In [1]:
from opt.mc_sim import *
from common.variables import *
from sim.sim_functions import *
import torch
import gym
from gym import spaces
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, Tuple, MultiBinary


/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
s = SourcingEnv()

In [3]:
class CustomGymEnv(Env):
        
    def __init__(self, sourcing_env):
        self.SourcingEnv = sourcing_env
        self.counter = 0
        
        # Actions we can take, down, stay, up
        self.action_space = MultiDiscrete([INVEN_LIMIT,INVEN_LIMIT])

        # Inventory Observation State
        self.observation_space = Box(low=np.array([-30, 0, 0, 0, 0]), high=np.array([30, 30, 30, 1, 1]), shape=(5,), dtype=int)
                                               
    
    def step(self, action):        
        reward = self.reward_func(self.SourcingEnv.current_state, action)
        next_state, event, i, event_probs, supplier_index = self.SourcingEnv.step(action)
        self.counter += 1
        
        info = {}
        
        if self.counter < PERIODS:
            done = False
        else:
            done = True
        
        next_state_array = np.array(next_state.get_list_repr())
        return next_state_array, reward, done, info
    
    def reset(self):
        self.SourcingEnv = SourcingEnv()
        return np.array(self.SourcingEnv.current_state.get_list_repr())
        
    def reward_func(self, state, action):
        reward_hb = H_COST * state.s if state.s >= 0 else (-B_PENALTY * state.s )
        reward = reward_hb + np.sum(np.multiply(action, PROCUREMENT_COST_VEC))
        reward = float(reward)
        return -reward

#INVEN_LIMIT

#Discrete(INVEN_LIMIT)

In [4]:
s = SourcingEnv()

In [5]:
custom_gym_env = CustomGymEnv(s)

In [6]:
m_state = custom_gym_env.SourcingEnv.current_state

str(m_state)

'Stock: 0, n backorders: [0. 0.], supplier status (on/off): [1. 1.]'

#PERIODS

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([2, 2]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
custom_gym_env.reset()

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [7]:
from stable_baselines3.common.env_checker import check_env

In [8]:
check_env(custom_gym_env) #warn=True)

episodes = 10
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    
    
    while not done:
        
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost=reward
        observation = custom_gym_env.step(action)
    print('Episode:{} Cost:{} Observation:{}' .format(episode, cost, observation))   #Observation: {}(, observation)
custom_gym_env.close()

In [9]:
from stable_baselines3 import A2C, DDPG, DQN, PPO, TD3, SAC
import os



In [43]:
env = custom_gym_env

In [44]:
log_path = os.path.join('Backup', 'A2C')

In [45]:
from stable_baselines3 import A2C
policy_kwargs = dict(net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
model = A2C("MlpPolicy", env, verbose=1,  learning_rate=(0.001),\
 n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \
tensorboard_log=log_path, seed=50, device='cuda' )
model.learn(total_timesteps=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to A2C/Training/Logs/A2C_1


model.learn(total_timesteps=10)

import gym
from stable_baselines3 import A2C
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
model = A2C("MlpPolicy", env, verbose=1,  learning_rate=linear_schedule(0.001),\ n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \ tensorboard_log=logdir,  policy_kwargs=policy_kwargs, seed=50, device='cuda' )
model.learn(total_timesteps=50000)

In [48]:
model.save('A2C')

In [49]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-390.0, 0.0)

custom_gym_env.observation_space

In [51]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [52]:
log_path = os.path.join('Backup','PPO')

In [53]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.01, device='cuda')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [54]:
model.learn(total_timesteps=10000)

Logging to PPO/Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -781     |
| time/              |          |
|    fps             | 1194     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -626      |
| time/                   |           |
|    fps                  | 971       |
|    iterations           | 2         |
|    time_elapsed         | 4         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0814244 |
|    clip_fraction        | 0.888     |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.73     |
|    explained_variance   | 1.19e-07  |
|    learning_rate       

In [55]:
model.save('PPO')

In [56]:
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

(-75.0, 0.0)

In [10]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 10, batch_size=50)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -609     |
| time/              |          |
|    fps             | 759      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 10       |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -643          |
| time/                   |               |
|    fps                  | 423           |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 20            |
| train/                  |               |
|    approx_kl            | 0.00059590937 |
|    clip_fraction        | 0             |
|    clip_

/usr/local/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:151: UserWarning: You have specified a mini-batch size of 50, but because the `RolloutBuffer` is of size `n_steps * n_envs = 10`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 10
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=10 and n_envs=1)
  warnings.warn(


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -606          |
| time/                   |               |
|    fps                  | 328           |
|    iterations           | 6             |
|    time_elapsed         | 0             |
|    total_timesteps      | 60            |
| train/                  |               |
|    approx_kl            | 0.00025086402 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.8          |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.0003        |
|    loss                 | 3.36e+05      |
|    n_updates            | 50            |
|    policy_gradient_loss | -0.00799      |
|    value_loss           | 6.73e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -671          |
| time/                   |               |
|    fps                  | 307           |
|    iterations           | 15            |
|    time_elapsed         | 0             |
|    total_timesteps      | 150           |
| train/                  |               |
|    approx_kl            | 0.00010425448 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.8          |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 4.06e+05      |
|    n_updates            | 140           |
|    policy_gradient_loss | -0.00521      |
|    value_loss           | 8.12e+05      |
-------------------------------------------
-----------------------------------------
| rollout/                |       

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -665          |
| time/                   |               |
|    fps                  | 304           |
|    iterations           | 24            |
|    time_elapsed         | 0             |
|    total_timesteps      | 240           |
| train/                  |               |
|    approx_kl            | 0.00016402602 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.79         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 1.91e+05      |
|    n_updates            | 230           |
|    policy_gradient_loss | -0.00724      |
|    value_loss           | 3.82e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

...

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 294      |
|    iterations           | 951      |
|    time_elapsed         | 32       |
|    total_timesteps      | 9510     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00197 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4e-15    |
|    n_updates            | 9500     |
|    policy_gradient_loss | 0        |
|    value_loss           | 1.6e-15  |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 294      |
|    iterations           | 962      |
|    time_elapsed         | 32       |
|    total_timesteps      | 9620     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00197 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4e-15    |
|    n_updates            | 9610     |
|    policy_gradient_loss | 0        |
|    value_loss           | 3.02e-15 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 294      |
|    iterations           | 973      |
|    time_elapsed         | 33       |
|    total_timesteps      | 9730     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00197 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4.44e-16 |
|    n_updates            | 9720     |
|    policy_gradient_loss | 0        |
|    value_loss           | 2.31e-15 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 294      |
|    iterations           | 984      |
|    time_elapsed         | 33       |
|    total_timesteps      | 9840     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00197 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4.44e-16 |
|    n_updates            | 9830     |
|    policy_gradient_loss | 0        |
|    value_loss           | 2.31e-15 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 294      |
|    iterations           | 995      |
|    time_elapsed         | 33       |
|    total_timesteps      | 9950     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00197 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4.44e-16 |
|    n_updates            | 9940     |
|    policy_gradient_loss | 0        |
|    value_loss           | 2.31e-15 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [11]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 10)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -682     |
| time/              |          |
|    fps             | 828      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 10       |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -713         |
| time/                   |              |
|    fps                  | 441          |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 20           |
| train/                  |              |
|    approx_kl            | 0.0006432772 |
|    clip_fraction        | 0            |
|    clip_range       

/usr/local/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:151: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 10`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 10
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=10 and n_envs=1)
  warnings.warn(


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -751          |
| time/                   |               |
|    fps                  | 327           |
|    iterations           | 7             |
|    time_elapsed         | 0             |
|    total_timesteps      | 70            |
| train/                  |               |
|    approx_kl            | 0.00026313664 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.8          |
|    explained_variance   | 1.79e-07      |
|    learning_rate        | 0.0003        |
|    loss                 | 3.56e+05      |
|    n_updates            | 60            |
|    policy_gradient_loss | -0.00869      |
|    value_loss           | 7.12e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -676          |
| time/                   |               |
|    fps                  | 311           |
|    iterations           | 16            |
|    time_elapsed         | 0             |
|    total_timesteps      | 160           |
| train/                  |               |
|    approx_kl            | 0.00021634698 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.8          |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 2.79e+05      |
|    n_updates            | 150           |
|    policy_gradient_loss | -0.00869      |
|    value_loss           | 5.58e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -679          |
| time/                   |               |
|    fps                  | 306           |
|    iterations           | 25            |
|    time_elapsed         | 0             |
|    total_timesteps      | 250           |
| train/                  |               |
|    approx_kl            | 0.00012142062 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.79         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 2.12e+05      |
|    n_updates            | 240           |
|    policy_gradient_loss | -0.00561      |
|    value_loss           | 4.25e+05      |
-------------------------------------------
------------------------------------------
| rollout/                |      

...

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                   |          |
|    fps                  | 298      |
|    iterations           | 953      |
|    time_elapsed         | 31       |
|    total_timesteps      | 9530     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00614 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 5.81e-09 |
|    n_updates            | 9520     |
|    policy_gradient_loss | 0        |
|    value_loss           | 1.25e-06 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                   |          |
|    fps                  | 298      |
|    iterations           | 964      |
|    time_elapsed         | 32       |
|    total_timesteps      | 9640     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00614 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 4.55e-13 |
|    n_updates            | 9630     |
|    policy_gradient_loss | 0        |
|    value_loss           | 1.66e-11 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                   |          |
|    fps                  | 298      |
|    iterations           | 975      |
|    time_elapsed         | 32       |
|    total_timesteps      | 9750     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00614 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 9740     |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                   |          |
|    fps                  | 298      |
|    iterations           | 986      |
|    time_elapsed         | 33       |
|    total_timesteps      | 9860     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00614 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 9850     |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                   |          |
|    fps                  | 298      |
|    iterations           | 997      |
|    time_elapsed         | 33       |
|    total_timesteps      | 9970     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00614 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 9960     |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -10      |
| time/                  

In [12]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 100)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -713     |
| time/              |          |
|    fps             | 1158     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/usr/local/lib/python3.8/site-packages/stable_baselines3/ppo/ppo.py:151: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 100`, after every 1 untruncated mini-batches, there will be a truncated mini-batch of size 36
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=100 and n_envs=1)
  warnings.warn(


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -700          |
| time/                   |               |
|    fps                  | 898           |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 200           |
| train/                  |               |
|    approx_kl            | 0.00032962504 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.8          |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 3.55e+05      |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.0118       |
|    value_loss           | 6.69e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -780        |
| time/                   |             |
|    fps                  | 760         |
|    iterations           | 11          |
|    time_elapsed         | 1           |
|    total_timesteps      | 1100        |
| train/                  |             |
|    approx_kl            | 0.000152358 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.79       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 2.92e+05    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00769    |
|    value_loss           | 5.63e+05    |
-----------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1 

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -562          |
| time/                   |               |
|    fps                  | 748           |
|    iterations           | 20            |
|    time_elapsed         | 2             |
|    total_timesteps      | 2000          |
| train/                  |               |
|    approx_kl            | 0.00018701004 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.77         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 3.43e+05      |
|    n_updates            | 190           |
|    policy_gradient_loss | -0.00886      |
|    value_loss           | 5.83e+05      |
-------------------------------------------
------------------------------------------
| rollout/                |      

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -11.6       |
| time/                   |             |
|    fps                  | 739         |
|    iterations           | 59          |
|    time_elapsed         | 7           |
|    total_timesteps      | 5900        |
| train/                  |             |
|    approx_kl            | 0.048418865 |
|    clip_fraction        | 0.285       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.95       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 362         |
|    n_updates            | 580         |
|    policy_gradient_loss | -0.082      |
|    value_loss           | 921         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1     

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -0.2         |
| time/                   |              |
|    fps                  | 738          |
|    iterations           | 69           |
|    time_elapsed         | 9            |
|    total_timesteps      | 6900         |
| train/                  |              |
|    approx_kl            | 0.0025691376 |
|    clip_fraction        | 0.0138       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.24        |
|    explained_variance   | 5.96e-08     |
|    learning_rate        | 0.0003       |
|    loss                 | 290          |
|    n_updates            | 680          |
|    policy_gradient_loss | 0.0266       |
|    value_loss           | 720          |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -0.05        |
| time/                   |              |
|    fps                  | 736          |
|    iterations           | 79           |
|    time_elapsed         | 10           |
|    total_timesteps      | 7900         |
| train/                  |              |
|    approx_kl            | 0.0011894347 |
|    clip_fraction        | 0.00651      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0588      |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 205          |
|    n_updates            | 780          |
|    policy_gradient_loss | -0.0102      |
|    value_loss           | 417          |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 734      |
|    iterations           | 89       |
|    time_elapsed         | 12       |
|    total_timesteps      | 8900     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.0184  |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 3.5e-05  |
|    n_updates            | 880      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0.000169 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 733      |
|    iterations           | 99       |
|    time_elapsed         | 13       |
|    total_timesteps      | 9900     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00638 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 2.41e-06 |
|    n_updates            | 980      |
|    policy_gradient_loss | 0        |
|    value_loss           | 9.28e-06 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [13]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 100, batch_size=50)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -677     |
| time/              |          |
|    fps             | 1148     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -652         |
| time/                   |              |
|    fps                  | 892          |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 200          |
| train/                  |              |
|    approx_kl            | 0.0003608173 |
|    clip_fraction        | 0            |
|    clip_range       

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | -677          |
| time/                   |               |
|    fps                  | 752           |
|    iterations           | 11            |
|    time_elapsed         | 1             |
|    total_timesteps      | 1100          |
| train/                  |               |
|    approx_kl            | 0.00016563357 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.79         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 2.8e+05       |
|    n_updates            | 100           |
|    policy_gradient_loss | -0.00809      |
|    value_loss           | 5.98e+05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -641         |
| time/                   |              |
|    fps                  | 740          |
|    iterations           | 20           |
|    time_elapsed         | 2            |
|    total_timesteps      | 2000         |
| train/                  |              |
|    approx_kl            | 0.0001932639 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -6.76        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 2.43e+05     |
|    n_updates            | 190          |
|    policy_gradient_loss | -0.00882     |
|    value_loss           | 4.86e+05     |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -528         |
| time/                   |              |
|    fps                  | 735          |
|    iterations           | 29           |
|    time_elapsed         | 3            |
|    total_timesteps      | 2900         |
| train/                  |              |
|    approx_kl            | 0.0007352042 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -6.67        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 2.12e+05     |
|    n_updates            | 280          |
|    policy_gradient_loss | -0.0177      |
|    value_loss           | 4.41e+05     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -300         |
| time/                   |              |
|    fps                  | 733          |
|    iterations           | 39           |
|    time_elapsed         | 5            |
|    total_timesteps      | 3900         |
| train/                  |              |
|    approx_kl            | 0.0051660566 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -6.26        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.35e+05     |
|    n_updates            | 380          |
|    policy_gradient_loss | -0.0424      |
|    value_loss           | 2.23e+05     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -5.2       |
| time/                   |            |
|    fps                  | 730        |
|    iterations           | 59         |
|    time_elapsed         | 8          |
|    total_timesteps      | 5900       |
| train/                  |            |
|    approx_kl            | 0.02409133 |
|    clip_fraction        | 0.125      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.41      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 225        |
|    n_updates            | 580        |
|    policy_gradient_loss | -0.0746    |
|    value_loss           | 453        |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -0.1         |
| time/                   |              |
|    fps                  | 729          |
|    iterations           | 69           |
|    time_elapsed         | 9            |
|    total_timesteps      | 6900         |
| train/                  |              |
|    approx_kl            | 0.0025666684 |
|    clip_fraction        | 0.011        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.206       |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 220          |
|    n_updates            | 680          |
|    policy_gradient_loss | -0.0188      |
|    value_loss           | 493          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 729           |
|    iterations           | 78            |
|    time_elapsed         | 10            |
|    total_timesteps      | 7800          |
| train/                  |               |
|    approx_kl            | 4.7683716e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0833       |
|    explained_variance   | nan           |
|    learning_rate        | 0.0003        |
|    loss                 | 55.5          |
|    n_updates            | 770           |
|    policy_gradient_loss | 0             |
|    value_loss           | 162           |
-------------------------------------------
--------------------------------------
| rollout/                |          

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | 0          |
| time/                   |            |
|    fps                  | 728        |
|    iterations           | 88         |
|    time_elapsed         | 12         |
|    total_timesteps      | 8800       |
| train/                  |            |
|    approx_kl            | 0.01192246 |
|    clip_fraction        | 0.018      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0155    |
|    explained_variance   | 1.19e-07   |
|    learning_rate        | 0.0003     |
|    loss                 | 0.449      |
|    n_updates            | 870        |
|    policy_gradient_loss | -0.019     |
|    value_loss           | 0.498      |
----------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean      

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 728      |
|    iterations           | 98       |
|    time_elapsed         | 13       |
|    total_timesteps      | 9800     |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00666 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 3.07e-08 |
|    n_updates            | 970      |
|    policy_gradient_loss | 0        |
|    value_loss           | 1.02e-07 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [14]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -734     |
| time/              |          |
|    fps             | 1176     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -736        |
| time/                   |             |
|    fps                  | 910         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.009811736 |
|    clip_fraction        | 0.0312      |
|    clip_range           | 0.2   

In [15]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -728     |
| time/              |          |
|    fps             | 1177     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -674        |
| time/                   |             |
|    fps                  | 912         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.008606156 |
|    clip_fraction        | 0.0185      |
|    clip_range           | 0.2   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -160       |
| time/                   |            |
|    fps                  | 769        |
|    iterations           | 11         |
|    time_elapsed         | 14         |
|    total_timesteps      | 11000      |
| train/                  |            |
|    approx_kl            | 0.03866457 |
|    clip_fraction        | 0.634      |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.44      |
|    explained_variance   | -3.58e-07  |
|    learning_rate        | 0.0003     |
|    loss                 | 4.96e+04   |
|    n_updates            | 100        |
|    policy_gradient_loss | -0.107     |
|    value_loss           | 7.65e+04   |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_m

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -9.65      |
| time/                   |            |
|    fps                  | 756        |
|    iterations           | 21         |
|    time_elapsed         | 27         |
|    total_timesteps      | 21000      |
| train/                  |            |
|    approx_kl            | 0.12875602 |
|    clip_fraction        | 0.909      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.62      |
|    explained_variance   | 5.96e-08   |
|    learning_rate        | 0.0003     |
|    loss                 | 67.9       |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.111     |
|    value_loss           | 408        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 752           |
|    iterations           | 31            |
|    time_elapsed         | 41            |
|    total_timesteps      | 31000         |
| train/                  |               |
|    approx_kl            | 0.00015566188 |
|    clip_fraction        | 0.0018        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.00713      |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 0.00139       |
|    n_updates            | 300           |
|    policy_gradient_loss | -0.00225      |
|    value_loss           | 2.05          |
-------------------------------------------
--------------------------------------
| rollout/                |          

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -0.45    |
| time/                   |          |
|    fps                  | 750      |
|    iterations           | 41       |
|    time_elapsed         | 54       |
|    total_timesteps      | 41000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00425 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 9.99e-14 |
|    n_updates            | 400      |
|    policy_gradient_loss | 0        |
|    value_loss           | 9.31e-06 |
--------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| tim

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 750      |
|    iterations           | 51       |
|    time_elapsed         | 67       |
|    total_timesteps      | 51000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00188 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 1.51e-15 |
|    n_updates            | 500      |
|    policy_gradient_loss | 0        |
|    value_loss           | 2.7e-15  |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 749      |
|    iterations           | 61       |
|    time_elapsed         | 81       |
|    total_timesteps      | 61000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00108 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 6.84e-15 |
|    n_updates            | 600      |
|    policy_gradient_loss | 0        |
|    value_loss           | 3.64e-15 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | 0            |
| time/                   |              |
|    fps                  | 748          |
|    iterations           | 71           |
|    time_elapsed         | 94           |
|    total_timesteps      | 71000        |
| train/                  |              |
|    approx_kl            | 0.0005018326 |
|    clip_fraction        | 0.0009       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.000489    |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 4.67e-05     |
|    n_updates            | 700          |
|    policy_gradient_loss | -0.00125     |
|    value_loss           | 0.0269       |
------------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean 

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |
| time/                   |           |
|    fps                  | 747       |
|    iterations           | 81        |
|    time_elapsed         | 108       |
|    total_timesteps      | 81000     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000359 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.78e-15  |
|    n_updates            | 800       |
|    policy_gradient_loss | 0         |
|    value_loss           | 3.55e-15  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |


--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 746      |
|    iterations           | 91       |
|    time_elapsed         | 121      |
|    total_timesteps      | 91000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00029 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 2.42e-14 |
|    n_updates            | 900      |
|    policy_gradient_loss | 0        |
|    value_loss           | 5.27e-14 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [16]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=10)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -688     |
| time/              |          |
|    fps             | 1173     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -639        |
| time/                   |             |
|    fps                  | 909         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.010088682 |
|    clip_fraction        | 0.0313      |
|    clip_range           | 0.2   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -185        |
| time/                   |             |
|    fps                  | 768         |
|    iterations           | 11          |
|    time_elapsed         | 14          |
|    total_timesteps      | 11000       |
| train/                  |             |
|    approx_kl            | 0.043221217 |
|    clip_fraction        | 0.594       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.37       |
|    explained_variance   | 1.19e-07    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.03e+04    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.104      |
|    value_loss           | 6.76e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1     

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -3.8      |
| time/                   |           |
|    fps                  | 755       |
|    iterations           | 21        |
|    time_elapsed         | 27        |
|    total_timesteps      | 21000     |
| train/                  |           |
|    approx_kl            | 0.1816574 |
|    clip_fraction        | 0.922     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.15     |
|    explained_variance   | -1.19e-07 |
|    learning_rate        | 0.0003    |
|    loss                 | 81.7      |
|    n_updates            | 200       |
|    policy_gradient_loss | -0.106    |
|    value_loss           | 1.24e+03  |
---------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -1.15    

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | 0            |
| time/                   |              |
|    fps                  | 754          |
|    iterations           | 31           |
|    time_elapsed         | 41           |
|    total_timesteps      | 31000        |
| train/                  |              |
|    approx_kl            | 0.0010853533 |
|    clip_fraction        | 0.0038       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0134      |
|    explained_variance   | 5.96e-08     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.00105      |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.00506     |
|    value_loss           | 0.177        |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | -0.05    |
| time/                   |          |
|    fps                  | 753      |
|    iterations           | 41       |
|    time_elapsed         | 54       |
|    total_timesteps      | 41000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00308 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 400      |
|    policy_gradient_loss | 0        |
|    value_loss           | 9.79e-15 |
--------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| tim

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 752      |
|    iterations           | 51       |
|    time_elapsed         | 67       |
|    total_timesteps      | 51000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00145 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 1.07e-13 |
|    n_updates            | 500      |
|    policy_gradient_loss | 0        |
|    value_loss           | 9.64e-06 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 751      |
|    iterations           | 61       |
|    time_elapsed         | 81       |
|    total_timesteps      | 61000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00138 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 600      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 749      |
|    iterations           | 72       |
|    time_elapsed         | 96       |
|    total_timesteps      | 72000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00138 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 710      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 747      |
|    iterations           | 83       |
|    time_elapsed         | 110      |
|    total_timesteps      | 83000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00138 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0        |
|    n_updates            | 820      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0        |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |
| time/                   |           |
|    fps                  | 747       |
|    iterations           | 94        |
|    time_elapsed         | 125       |
|    total_timesteps      | 94000     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000877 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.26e-11  |
|    n_updates            | 930       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.000687  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |


In [17]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=100)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -773     |
| time/              |          |
|    fps             | 1194     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -630        |
| time/                   |             |
|    fps                  | 924         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.009022917 |
|    clip_fraction        | 0.0165      |
|    clip_range           | 0.2   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -194        |
| time/                   |             |
|    fps                  | 770         |
|    iterations           | 11          |
|    time_elapsed         | 14          |
|    total_timesteps      | 11000       |
| train/                  |             |
|    approx_kl            | 0.036753573 |
|    clip_fraction        | 0.548       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.44       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 4.46e+04    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.102      |
|    value_loss           | 8.2e+04     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1     

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -6.6       |
| time/                   |            |
|    fps                  | 757        |
|    iterations           | 21         |
|    time_elapsed         | 27         |
|    total_timesteps      | 21000      |
| train/                  |            |
|    approx_kl            | 0.17574306 |
|    clip_fraction        | 0.785      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.65      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 8e+03      |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.111     |
|    value_loss           | 1.13e+03   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1            |
|    ep_rew_mean          | -0.1         |
| time/                   |              |
|    fps                  | 752          |
|    iterations           | 31           |
|    time_elapsed         | 41           |
|    total_timesteps      | 31000        |
| train/                  |              |
|    approx_kl            | 0.0010217164 |
|    clip_fraction        | 0.0089       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.0394      |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 0.86         |
|    n_updates            | 300          |
|    policy_gradient_loss | 0.481        |
|    value_loss           | 1.04         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 750           |
|    iterations           | 41            |
|    time_elapsed         | 54            |
|    total_timesteps      | 41000         |
| train/                  |               |
|    approx_kl            | 2.0802022e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.00941      |
|    explained_variance   | nan           |
|    learning_rate        | 0.0003        |
|    loss                 | 0.156         |
|    n_updates            | 400           |
|    policy_gradient_loss | 0.157         |
|    value_loss           | 1.49e-05      |
-------------------------------------------
-------------------------------------------
| rollout/                |     

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 748      |
|    iterations           | 51       |
|    time_elapsed         | 68       |
|    total_timesteps      | 51000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00347 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 2.42e-14 |
|    n_updates            | 500      |
|    policy_gradient_loss | 0        |
|    value_loss           | 4.22e-14 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 746      |
|    iterations           | 61       |
|    time_elapsed         | 81       |
|    total_timesteps      | 61000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00235 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 2.46e-13 |
|    n_updates            | 600      |
|    policy_gradient_loss | 0        |
|    value_loss           | 9.47e-06 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |
| time/                   |           |
|    fps                  | 745       |
|    iterations           | 71        |
|    time_elapsed         | 95        |
|    total_timesteps      | 71000     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000767 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.11e-14  |
|    n_updates            | 700       |
|    policy_gradient_loss | 0         |
|    value_loss           | 6.14e-14  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |
| time/                   |           |
|    fps                  | 744       |
|    iterations           | 81        |
|    time_elapsed         | 108       |
|    total_timesteps      | 81000     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000682 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 1.11e-14  |
|    n_updates            | 800       |
|    policy_gradient_loss | 0         |
|    value_loss           | 1.38e-14  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |
| time/                   |           |
|    fps                  | 743       |
|    iterations           | 91        |
|    time_elapsed         | 122       |
|    total_timesteps      | 91000     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000682 |
|    explained_variance   | nan       |
|    learning_rate        | 0.0003    |
|    loss                 | 4e-15     |
|    n_updates            | 900       |
|    policy_gradient_loss | 0         |
|    value_loss           | 9.76e-14  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | 0         |


In [18]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=1000)
model.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Backup/PPO/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -728     |
| time/              |          |
|    fps             | 1189     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -650        |
| time/                   |             |
|    fps                  | 925         |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 2000        |
| train/                  |             |
|    approx_kl            | 0.009310101 |
|    clip_fraction        | 0.0299      |
|    clip_range           | 0.2   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -155       |
| time/                   |            |
|    fps                  | 778        |
|    iterations           | 11         |
|    time_elapsed         | 14         |
|    total_timesteps      | 11000      |
| train/                  |            |
|    approx_kl            | 0.04282795 |
|    clip_fraction        | 0.571      |
|    clip_range           | 0.2        |
|    entropy_loss         | -5.4       |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 4.54e+04   |
|    n_updates            | 100        |
|    policy_gradient_loss | -0.106     |
|    value_loss           | 8.05e+04   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -4.6       |
| time/                   |            |
|    fps                  | 765        |
|    iterations           | 21         |
|    time_elapsed         | 27         |
|    total_timesteps      | 21000      |
| train/                  |            |
|    approx_kl            | 0.11813905 |
|    clip_fraction        | 0.927      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.39      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 48.6       |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.122     |
|    value_loss           | 278        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 760           |
|    iterations           | 31            |
|    time_elapsed         | 40            |
|    total_timesteps      | 31000         |
| train/                  |               |
|    approx_kl            | 0.00014822895 |
|    clip_fraction        | 0.0017        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0199       |
|    explained_variance   | 1.79e-07      |
|    learning_rate        | 0.0003        |
|    loss                 | 0.000661      |
|    n_updates            | 300           |
|    policy_gradient_loss | -0.00248      |
|    value_loss           | 19.1          |
-------------------------------------------
-------------------------------------------
| rollout/                |     

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 758      |
|    iterations           | 41       |
|    time_elapsed         | 54       |
|    total_timesteps      | 41000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00344 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 1.08e-13 |
|    n_updates            | 400      |
|    policy_gradient_loss | 0        |
|    value_loss           | 3.24e-06 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 756           |
|    iterations           | 51            |
|    time_elapsed         | 67            |
|    total_timesteps      | 51000         |
| train/                  |               |
|    approx_kl            | 0.00013561524 |
|    clip_fraction        | 0.0009        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.00287      |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | 0.00241       |
|    n_updates            | 500           |
|    policy_gradient_loss | -0.00125      |
|    value_loss           | 50.6          |
-------------------------------------------
--------------------------------------
| rollout/                |          

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 754      |
|    iterations           | 61       |
|    time_elapsed         | 80       |
|    total_timesteps      | 61000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00186 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 3.73e-14 |
|    n_updates            | 600      |
|    policy_gradient_loss | 0        |
|    value_loss           | 7.46e-14 |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | 0          |
| time/                   |            |
|    fps                  | 753        |
|    iterations           | 71         |
|    time_elapsed         | 94         |
|    total_timesteps      | 71000      |
| train/                  |            |
|    approx_kl            | 0.01570997 |
|    clip_fraction        | 0.0279     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0638    |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 179        |
|    n_updates            | 700        |
|    policy_gradient_loss | -0.0289    |
|    value_loss           | 484        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 1             |
|    ep_rew_mean          | 0             |
| time/                   |               |
|    fps                  | 752           |
|    iterations           | 81            |
|    time_elapsed         | 107           |
|    total_timesteps      | 81000         |
| train/                  |               |
|    approx_kl            | 0.00012567961 |
|    clip_fraction        | 0.0008        |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.00813      |
|    explained_variance   | 0             |
|    learning_rate        | 0.0003        |
|    loss                 | -0.425        |
|    n_updates            | 800           |
|    policy_gradient_loss | -0.405        |
|    value_loss           | 8.11          |
-------------------------------------------
--------------------------------------
| rollout/                |          

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 751      |
|    iterations           | 91       |
|    time_elapsed         | 121      |
|    total_timesteps      | 91000    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00372 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0.00256  |
|    n_updates            | 900      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0.00292  |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [58]:

torch.cuda.is_available()

False

In [59]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [60]:
 torch.cuda.device_count()

0